In [2]:
from torchvision.datasets import CIFAR10
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [6]:
class CIFAR10Dataset(Dataset):
    def __init__(self, is_train: bool):
        normalization = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ]
        )
        dataset = CIFAR10(root="data", download=True, train=is_train, transform=normalization)
        dataset.data = dataset.data[:100]
        dataset.targets = dataset.targets[:100]

        dataloader = DataLoader(dataset)

        self.data = torch.cat([X for X, _ in dataloader])
        self.labels = torch.eye(10)[torch.cat([y for _, y in dataloader])]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.labels[index]
        return x, y


train_dataset = CIFAR10Dataset(True)

Files already downloaded and verified
